In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

exported_files_dir = './monte_carlo'

# Load the data
df = pd.read_csv('population_stats.csv')

# Add a column for population group (Chinese, Finnish, Italian)
df['Group'] = df['Population'].apply(lambda x: x.split()[0])

# Display basic information about the dataset
print("Dataset Information:")
print(f"Total samples: {len(df)}")
print(f"Populations: {df['Group'].unique()}") # Just in case we need other countries
print(f"SNPs analyzed: rs762551, rs2472304, rs2470893")
print("\n")

# Calculate basic statistics by population group
group_stats = df.groupby('Group').agg({
    '# Variants': 'mean',
    'SNPs': 'mean',
    'Insertions': 'mean',
    'Deletions': 'mean'
}).reset_index()

print("Average genetic variations by population:")
print(group_stats.round(2))
print("\n")

# Function to calculate SNP variant frequencies
def calculate_snp_frequency(population_group, snp_column):
    '''
        Import a population and the name of the snp to calculate
    '''
    population_data = df[df['Group'] == population_group]
    variant_counts = population_data[snp_column].value_counts()
    variant_percentages = (variant_counts / len(population_data) * 100).round(1)
    return variant_percentages

# Analyze rs762551 SNP across populations
print("rs762551 SNP distribution:")
for group in df['Group'].unique():
    print(f"\n{group}:")
    print(calculate_snp_frequency(group, 'rs762551'))

# Analyze rs2472304 SNP across populations
print("\nrs2472304 SNP distribution:")
for group in df['Group'].unique():
    print(f"\n{group}:")
    print(calculate_snp_frequency(group, 'rs2472304'))

# Analyze rs2470893 SNP (only found in Finnish)
print("\nrs2470893 SNP distribution:")
for group in df['Group'].unique():
    print(f"\n{group}:")
    print(calculate_snp_frequency(group, 'rs2470893'))

# Create a function to check if a SNP variant is found
def is_snp_found(variant):
    if pd.isna(variant) or variant == 'not found' or variant == 'not found ': #Sometimes there's a space after not found
        return 0
    else:
        return 1

# Calculate the percentage of samples where each SNP is found
snp_columns = ['rs762551', 'rs2472304', 'rs2470893']
snp_found_data = []

for group in df['Group'].unique():
    group_data = df[df['Group'] == group]
    
    for snp in snp_columns:
        found_percentage = (group_data[snp].apply(is_snp_found).sum() / len(group_data) * 100).round(1)
        snp_found_data.append({
            'Group': group,
            'SNP': snp,
            'Found (%)': found_percentage
        })

snp_found_df = pd.DataFrame(snp_found_data)
print("\nPercentage of samples where SNPs were found:")
print(pd.pivot_table(snp_found_df, values='Found (%)', index='Group', columns='SNP'))

# Function to extract genotype information
def extract_genotype(variant):
    if pd.isna(variant) or 'not found' in str(variant):
        return 'Not Found'
    if 'homozygous' in str(variant):
        return 'Homozygous'
    if 'heterozygous' in str(variant):
        return 'Heterozygous'
    return 'Other'

# Add genotype columns for the key SNPs
df['rs762551_genotype'] = df['rs762551'].apply(extract_genotype)
df['rs2472304_genotype'] = df['rs2472304'].apply(extract_genotype)

# Summary statistics for rs762551 by population group
rs762551_summary = pd.crosstab(
    df['Group'], 
    df['rs762551_genotype'], 
    normalize='index'
) * 100

print("\nrs762551 genotype distribution (%):")
print(rs762551_summary.round(1))

# Summary statistics for rs2472304 by population group
rs2472304_summary = pd.crosstab(
    df['Group'], 
    df['rs2472304_genotype'], 
    normalize='index'
) * 100

print("\nrs2472304 genotype distribution (%):")
print(rs2472304_summary.round(1))

# Create visualizations

# Set the style for the plots
plt.style.use('seaborn-v0_8-whitegrid')

# 1. Visualization for rs762551 SNP
plt.figure(figsize=(12, 6))

# Convert crosstab to format suitable for grouped bar chart
rs762551_plot_data = rs762551_summary.reset_index()
rs762551_plot_data_melted = pd.melt(
    rs762551_plot_data, 
    id_vars='Group', 
    value_vars=['Homozygous', 'Heterozygous', 'Not Found'],
    var_name='Genotype', 
    value_name='Percentage'
)

# Create the grouped bar chart
ax1 = sns.barplot(x='Group', y='Percentage', hue='Genotype', data=rs762551_plot_data_melted)
plt.title('rs762551 Genotype Distribution by Population', fontsize=14)
plt.xlabel('Population Group', fontsize=12)
plt.ylabel('Percentage (%)', fontsize=12)
plt.legend(title='Genotype')
plt.savefig(f'{exported_files_dir}/rs762551_distribution.png', dpi=300, bbox_inches='tight')

# 2. Visualization for rs2472304 SNP
plt.figure(figsize=(12, 6))

# Convert crosstab to format suitable for grouped bar chart
rs2472304_plot_data = rs2472304_summary.reset_index()
rs2472304_plot_data_melted = pd.melt(
    rs2472304_plot_data, 
    id_vars='Group', 
    value_vars=['Homozygous', 'Heterozygous', 'Not Found'],
    var_name='Genotype', 
    value_name='Percentage'
)

# Create the grouped bar chart
ax2 = sns.barplot(x='Group', y='Percentage', hue='Genotype', data=rs2472304_plot_data_melted)
plt.title('rs2472304 Genotype Distribution by Population', fontsize=14)
plt.xlabel('Population Group', fontsize=12)
plt.ylabel('Percentage (%)', fontsize=12)
plt.legend(title='Genotype')
plt.savefig(f'{exported_files_dir}/rs2472304_distribution.png', dpi=300, bbox_inches='tight')

# 3. Heatmap of SNP presence across populations
plt.figure(figsize=(10, 6))
pivot_data = pd.pivot_table(snp_found_df, values='Found (%)', index='Group', columns='SNP')
sns.heatmap(pivot_data, annot=True, cmap='YlGnBu', fmt='.1f')
plt.title('Percentage of Samples with SNP Variants by Population', fontsize=14)
plt.tight_layout()
plt.savefig(f'{exported_files_dir}/snp_presence_heatmap.png', dpi=300, bbox_inches='tight')

# 4. Average genetic variations by population
plt.figure(figsize=(12, 6))
melted_stats = pd.melt(
    group_stats, 
    id_vars='Group', 
    value_vars=['# Variants', 'SNPs', 'Insertions', 'Deletions'],
    var_name='Variation Type', 
    value_name='Average Count'
)
sns.barplot(x='Group', y='Average Count', hue='Variation Type', data=melted_stats)
plt.title('Average Genetic Variations by Population', fontsize=14)
plt.xlabel('Population Group', fontsize=12)
plt.ylabel('Average Count', fontsize=12)
plt.legend(title='Variation Type')
plt.savefig(f'{exported_files_dir}/average_variations.png', dpi=300, bbox_inches='tight')

print("\nAnalysis complete. Visualization files have been saved.")

print("\n3. Other SNPs:")
print("   - rs2069514 and rs2472300 were not found in any population samples")
print("   - rs2470893 was only found in some Finnish samples as a heterozygous variant")

print("\n4. Overall genetic variation:")
groupby_stats = df.groupby('Group')[['# Variants', 'SNPs']].mean().round(1)
for group, row in groupby_stats.iterrows():
    print(f"   - {group} population: {row['# Variants']} variants and {row['SNPs']} SNPs on average")

# Add Monte Carlo analysis section
print("\n\n================================")
print("MONTE CARLO STATISTICAL ANALYSIS")
print("================================")

# Function to visualize Monte Carlo results
def plot_monte_carlo_distribution(observed_statistic, perm_statistics, statistic_name, title):
    """
    Create a histogram of permutation statistics with the observed statistic highlighted.
    """
    plt.figure(figsize=(10, 6))
    
    # Plot histogram of permutation statistics
    sns.histplot(perm_statistics, bins=30, kde=True)
    
    # Add line for observed statistic
    plt.axvline(observed_statistic, color='red', linestyle='--', 
                label=f'Observed {statistic_name} = {observed_statistic:.4f}')
    
    # Calculate p-value
    p_value = sum(1 for x in perm_statistics if x >= observed_statistic) / len(perm_statistics)
    
    plt.title(f"{title}\nP-value = {p_value:.4f}")
    plt.xlabel(statistic_name)
    plt.ylabel('Frequency')
    plt.legend()
    
    # Save figure
    filename = f"{exported_files_dir}/monte_carlo_{title.replace(' ', '_').replace(':', '').lower()}.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved Monte Carlo visualization: {filename}")

def monte_carlo_genotype_test(df, snp_column, n_permutations=10000):
    """
    Perform Monte Carlo permutation test to calculate p-values for differences
    in genotype distributions between populations.
    """
    # Extract population groups
    groups = df['Group'].unique()
    
    # Calculate observed chi-square statistics between groups
    observed_chi2 = {}
    for i, group1 in enumerate(groups):
        for group2 in groups[i+1:]:
            # Get genotype distributions for each group
            group1_data = df[df['Group'] == group1]
            group2_data = df[df['Group'] == group2]
            
            # Count genotypes in each group
            group1_counts = group1_data[snp_column].value_counts().to_dict()
            group2_counts = group2_data[snp_column].value_counts().to_dict()
            
            # Combine all genotypes
            all_genotypes = set(list(group1_counts.keys()) + list(group2_counts.keys()))
            
            # Calculate chi-square statistic
            chi2 = 0
            for genotype in all_genotypes:
                count1 = group1_counts.get(genotype, 0)
                count2 = group2_counts.get(genotype, 0)
                
                # Expected proportions based on combined data
                total_count = count1 + count2
                if total_count > 0:
                    expected1 = total_count * (len(group1_data) / (len(group1_data) + len(group2_data)))
                    expected2 = total_count * (len(group2_data) / (len(group1_data) + len(group2_data)))
                    
                    # Add to chi-square if expected counts are non-zero
                    if expected1 > 0:
                        chi2 += ((count1 - expected1) ** 2) / expected1
                    if expected2 > 0:
                        chi2 += ((count2 - expected2) ** 2) / expected2
            
            observed_chi2[(group1, group2)] = chi2
    
    # Perform permutation test
    perm_chi2 = {pair: [] for pair in observed_chi2.keys()}
    
    for _ in range(n_permutations):
        # Shuffle the population labels
        shuffled_df = df.copy()
        shuffled_df['Group'] = np.random.permutation(df['Group'].values)
        
        # Calculate chi-square with shuffled labels
        for (group1, group2) in observed_chi2.keys():
            # Get genotype distributions for each shuffled group
            group1_data = shuffled_df[shuffled_df['Group'] == group1]
            group2_data = shuffled_df[shuffled_df['Group'] == group2]
            
            # Count genotypes in each group
            group1_counts = group1_data[snp_column].value_counts().to_dict()
            group2_counts = group2_data[snp_column].value_counts().to_dict()
            
            # Combine all genotypes
            all_genotypes = set(list(group1_counts.keys()) + list(group2_counts.keys()))
            
            # Calculate chi-square statistic
            chi2 = 0
            for genotype in all_genotypes:
                count1 = group1_counts.get(genotype, 0)
                count2 = group2_counts.get(genotype, 0)
                
                # Expected proportions based on combined data
                total_count = count1 + count2
                if total_count > 0:
                    expected1 = total_count * (len(group1_data) / (len(group1_data) + len(group2_data)))
                    expected2 = total_count * (len(group2_data) / (len(group1_data) + len(group2_data)))
                    
                    # Add to chi-square if expected counts are non-zero
                    if expected1 > 0:
                        chi2 += ((count1 - expected1) ** 2) / expected1
                    if expected2 > 0:
                        chi2 += ((count2 - expected2) ** 2) / expected2
            
            perm_chi2[(group1, group2)].append(chi2)
    
    # Calculate p-values and create visualizations
    p_values = {}
    for pair, obs_chi2 in observed_chi2.items():
        # Count how many permutations had a chi-square >= observed
        group1, group2 = pair
        count = sum(1 for perm_c2 in perm_chi2[pair] if perm_c2 >= obs_chi2)
        p_value = count / n_permutations
        p_values[pair] = p_value
        
        # Create visualization
        plot_monte_carlo_distribution(
            obs_chi2, 
            perm_chi2[pair], 
            "Chi-Square Statistic", 
            f"{snp_column.replace('_genotype', '')} Genotype {group1} vs {group2}"
        )
    
    # Format results as a DataFrame
    result_data = []
    for (group1, group2), p_value in p_values.items():
        result_data.append({
            'Group 1': group1,
            'Group 2': group2,
            'Chi-Square Statistic': observed_chi2[(group1, group2)],
            'P-value': p_value
        })
    
    return pd.DataFrame(result_data)

def monte_carlo_allele_test(df, snp_column, n_permutations=10000):
    """
    Perform Monte Carlo permutation test to calculate p-values for differences
    in allele frequencies between populations.
    """
    # Extract population groups
    groups = df['Group'].unique()
    
    # Calculate observed differences in allele frequencies between groups
    observed_diffs = {}
    for i, group1 in enumerate(groups):
        for group2 in groups[i+1:]:
            # Calculate allele frequencies for each group
            group1_data = df[df['Group'] == group1]
            group2_data = df[df['Group'] == group2]
            
            # Simple presence/absence frequency
            group1_freq = group1_data[snp_column].apply(is_snp_found).mean()
            group2_freq = group2_data[snp_column].apply(is_snp_found).mean()
            
            # Store the absolute difference
            observed_diffs[(group1, group2)] = abs(group1_freq - group2_freq)
    
    # Perform permutation test
    perm_diffs = {pair: [] for pair in observed_diffs.keys()}
    
    for _ in range(n_permutations):
        # Shuffle the population labels
        shuffled_df = df.copy()
        shuffled_df['Group'] = np.random.permutation(df['Group'].values)
        
        # Calculate differences with shuffled labels
        for (group1, group2) in observed_diffs.keys():
            group1_data = shuffled_df[shuffled_df['Group'] == group1]
            group2_data = shuffled_df[shuffled_df['Group'] == group2]
            
            group1_freq = group1_data[snp_column].apply(is_snp_found).mean()
            group2_freq = group2_data[snp_column].apply(is_snp_found).mean()
            
            perm_diffs[(group1, group2)].append(abs(group1_freq - group2_freq))
    
    # Calculate p-values and create visualizations
    p_values = {}
    for pair, obs_diff in observed_diffs.items():
        group1, group2 = pair
        # Count how many permutations had a difference >= observed
        count = sum(1 for perm_diff in perm_diffs[pair] if perm_diff >= obs_diff)
        p_value = count / n_permutations
        p_values[pair] = p_value
        
        # Create visualization
        plot_monte_carlo_distribution(
            obs_diff, 
            perm_diffs[pair], 
            "Absolute Difference in Allele Frequency", 
            f"{snp_column} Frequency {group1} vs {group2}"
        )
    
    # Format results as a DataFrame
    result_data = []
    for (group1, group2), p_value in p_values.items():
        result_data.append({
            'Group 1': group1,
            'Group 2': group2,
            'Observed Frequency Difference': observed_diffs[(group1, group2)],
            'P-value': p_value
        })
    
    return pd.DataFrame(result_data)

# Number of permutations for Monte Carlo analysis
n_permutations = 10000
print(f"\nPerforming Monte Carlo analysis with {n_permutations} permutations...\n")

# Apply Monte Carlo analysis for genotype distributions
print("\nMonte Carlo Analysis for Differences in Genotype Distributions:")
genotype_pvalues = {}
for snp in ['rs762551_genotype', 'rs2472304_genotype']:
    base_snp = snp.replace('_genotype', '')
    print(f"\nP-values for {base_snp} genotype distributions:")
    p_values_df = monte_carlo_genotype_test(df, snp, n_permutations)
    genotype_pvalues[base_snp] = p_values_df
    print(p_values_df.round(4))

# Apply Monte Carlo analysis for allele frequencies
print("\nMonte Carlo Analysis for Differences in Allele Frequencies:")
allele_pvalues = {}
for snp in ['rs762551', 'rs2472304', 'rs2470893']:
    print(f"\nP-values for {snp} allele frequencies:")
    p_values_df = monte_carlo_allele_test(df, snp, n_permutations)
    allele_pvalues[snp] = p_values_df
    print(p_values_df.round(4))

# Create summary table of p-values for all comparisons
print("\nSummary of Monte Carlo p-values across all SNP comparisons:")
summary_data = []

# Get all unique population pairs
all_pairs = []
for result_df in genotype_pvalues.values():
    for _, row in result_df.iterrows():
        pair = (row['Group 1'], row['Group 2'])
        if pair not in all_pairs:
            all_pairs.append(pair)

# Collect p-values for each SNP and comparison
for pair in all_pairs:
    group1, group2 = pair
    row_data = {'Group 1': group1, 'Group 2': group2}
    
    # Add genotype p-values
    for snp, result_df in genotype_pvalues.items():
        match_row = result_df[(result_df['Group 1'] == group1) & (result_df['Group 2'] == group2)]
        if not match_row.empty:
            row_data[f"{snp} (genotype)"] = match_row['P-value'].values[0]
    
    # Add allele frequency p-values
    for snp, result_df in allele_pvalues.items():
        match_row = result_df[(result_df['Group 1'] == group1) & (result_df['Group 2'] == group2)]
        if not match_row.empty:
            row_data[f"{snp} (frequency)"] = match_row['P-value'].values[0]
    
    summary_data.append(row_data)

summary_df = pd.DataFrame(summary_data)
print(summary_df.round(4))

# Create heatmap of p-values
print("\nCreating heatmap visualization of p-values...")

# Reshape data for heatmap
test_types = []
for snp in genotype_pvalues.keys():
    test_types.append(f"{snp} (genotype)")
for snp in allele_pvalues.keys():
    test_types.append(f"{snp} (frequency)")

heatmap_data = []
for pair in all_pairs:
    group1, group2 = pair
    pair_label = f"{group1} vs {group2}"
    
    for test in test_types:
        p_value = None
        
        # Find the p-value in our summary data
        for row in summary_data:
            if row['Group 1'] == group1 and row['Group 2'] == group2 and test in row:
                p_value = row[test]
        
        if p_value is not None:
            heatmap_data.append({
                'Comparison': pair_label,
                'Test': test,
                'P-value': p_value
            })

heatmap_df = pd.DataFrame(heatmap_data)
heatmap_pivot = heatmap_df.pivot(index='Comparison', columns='Test', values='P-value')

# Create the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_pivot, annot=True, cmap='YlOrRd_r', fmt='.4f', 
            vmin=0, vmax=0.1, cbar_kws={'label': 'P-value'})
plt.title('Monte Carlo P-values Across Population Comparisons', fontsize=14)
plt.tight_layout()
plt.savefig(f'{exported_files_dir}/monte_carlo_pvalues_heatmap.png', dpi=300, bbox_inches='tight')

print("\nMonte Carlo analysis complete. Visualizations and summary have been saved.")

# Add interpretation of results based on significance threshold
print("\nInterpretation of Monte Carlo Results:")
print("Statistical significance is typically declared at p < 0.05.")
significant_results = heatmap_df[heatmap_df['P-value'] < 0.05]

if len(significant_results) > 0:
    print("\nStatistically significant differences were found in:")
    for _, row in significant_results.iterrows():
        print(f"- {row['Comparison']} for {row['Test']}: p = {row['P-value']:.4f}")
    
    # Group significant findings by population comparison
    for pair in all_pairs:
        group1, group2 = pair
        pair_label = f"{group1} vs {group2}"
        
        pair_results = significant_results[significant_results['Comparison'] == pair_label]
        if len(pair_results) > 0:
            print(f"\nSignificant differences between {group1} and {group2}:")
            for _, row in pair_results.iterrows():
                test_parts = row['Test'].split()
                snp = test_parts[0]
                test_type = test_parts[1].strip('()')
                
                print(f"  * {snp} {test_type}: p = {row['P-value']:.4f}")
            print(f"  This suggests the {snp} allele distribution differs significantly between these populations.")
else:
    print("\nNo statistically significant differences were found at p < 0.05 threshold.")
    
    # Find the lowest p-values
    min_p = heatmap_df['P-value'].min()
    min_rows = heatmap_df[heatmap_df['P-value'] == min_p]
    
    print(f"\nThe lowest p-value was {min_p:.4f}, found in:")
    for _, row in min_rows.iterrows():
        print(f"- {row['Comparison']} for {row['Test']}")

Traceback (most recent call last):
  File "/Users/matthewkinder/.vscode/extensions/ms-python.python-2025.2.0-darwin-arm64/python_files/python_server.py", line 133, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'pandas'

